<a href="https://colab.research.google.com/github/Athugodage/RuLawSimplification/blob/main/T5/fkgl_for_t5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.9 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

text_generator = pipeline("text2text-generation", 
                          model='marcus2000/T5-RLS2000')

In [ ]:
!git clone https://github.com/Athugodage/RuLawSimplification.git

Cloning into 'RuLawSimplification'...
remote: Enumerating objects: 557, done.
remote: Counting objects: 100% (291/291), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 557 (delta 153), reused 152 (delta 79), pack-reused 266
Receiving objects: 100% (557/557), 244.03 MiB | 19.93 MiB/s, done.
Resolving deltas: 100% (266/266), done.


In [ ]:
!unzip /content/RuLawSimplification/corpus/the_corpus.zip

Archive:  /content/RuLawSimplification/corpus/the_corpus.zip
  inflating: the_corpus.csv          


In [ ]:
import pandas as pd

df = pd.read_csv('the_corpus.csv')

In [ ]:
df = df.drop(['Название документа', 'Ссылка', 'Date'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, random_state=42, test_size=0.05)

In [ ]:
train.columns = ["text", "commentary"]
test.columns = ["text", "commentary"]

In [ ]:
print(text_generator([test.iloc[0]['text'][:2500]])[0]['generated_text'])

В законе также прописаны требования к организации кадастровой деятельности.


In [ ]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
a = ['a', 'b', 'c']
z = ''
for aa in a:
  z = z + aa

print(z)

abc


In [ ]:
from tqdm import tqdm
from razdel import sentenize

real = test['text'].to_list()
preds = []

for orig in tqdm(real):
    sents =  list(sentenize(orig))
    generated_text = ''

    for sent in sents:
      generated_sent = text_generator(sent.text)
      generated_sent = str(generated_sent[0]['generated_text'])
      generated_text = generated_text + generated_sent

    preds.append(generated_text)

 31%|███▏      | 32/102 [4:21:13<9:31:25, 489.79s/it]  


KeyboardInterrupt: ignored

In [ ]:
## GUNNING FOG READABILITY INDEX

def gunning_fog(text):
  text = str(text)

  tokens = nltk.word_tokenize(text)
  tokens_length = len(tokens)
  sent_length = len(nltk.sent_tokenize(text))
  complex_tokens_len = len([tok for tok in tokens if len(tok) > 6])
  formule = 0.4 * (tokens_length / sent_length) + 100 * (complex_tokens_len / tokens_length)
  return formule

In [ ]:
import nltk
import numpy as np
nltk.download('punkt')


indexes = []

## original

for t in test['text'].to_list():
  indexes.append(gunning_fog(t))

print(np.mean(indexes))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


53.29907835008921


In [ ]:
indexes = []

## simplified by model

for pred in preds:
  indexes.append(gunning_fog(pred))

print(np.mean(indexes))

548.772278104454


FKGL= 0.39 (total words / total sentences)+11.8 (total syllabl/total words)-15.59

In [ ]:
from razdel import sentenize, tokenize


def all_syl_number(string):
    num_vowels = 0

    for char in string.lower():
        if char in "яыуаеиоюэ":
            num_vowels = num_vowels+1
       
    return num_vowels

def total_words(string):
    return len(list(tokenize(string)))

def total_sents(string):
    return len(list(sentenize(string)))
    
def fkgl(string):
    n_of_sents = total_sents(string)
    n_of_words = total_words(string)
    n_of_syls = all_syl_number(string)
    fkgl = (0.39 * (n_of_words/n_of_sents) + 11.8 * (n_of_syls/n_of_words)) - 15.39
    return float(fkgl)


In [ ]:
# FKGL

indexes = []

## original

for t in test['text'].to_list():
  indexes.append(fkgl(t))

print(np.mean(indexes))

26.582805927512325


In [ ]:
import re
new_preds = []

for pred in preds:
  new = re.sub(r'/w/W', r'/w./s/W', pred)
  new = re.sub(r'/w./W', r'/w./s/W', new)
  new_preds.append(new)

In [ ]:
new_preds[0]

'В законе также прописаны требования к кадастровой стоимости.В этом году в России будет создано более 450 тысяч новых рабочихВ этом году в России будет создано более 450 тысяч новых рабочихВ этом году в России будет действовать новый порядок предоставления субсидий.В этом году в России будет создано более 450 тысяч новых рабочихВ частности, вводится запрет на проведение массовых мероприятий.Если же кадастровые работы выполняются для собственных нужд юридического лица, тоНо в этом случае договор подряда не требуется.В частности, в отношении земельных участков, которые не подлежат государственной кадастровойВ случае выявления незавершенных объектов, в том числе в границахВ частности, в документе прописаны требования к кадастровой стоимости,В частности, в рамках комплексного освоения территории, в том числе вВ частности, в границах, которые были включены в комплексное развитиеВ частности, в границах, которые были выделены под комплексное развитиеВ частности, в отношении земельных участков

In [ ]:
indexes = []

## simplified by model

for pred in preds:
  indexes.append(fkgl(pred))

print(np.mean(indexes))

535.4929107351658
